In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x40d9ae0
Welcome to JupyROOT 6.26/10


In [2]:
ROOT.EnableImplicitMT(4)

In [3]:
%jsroot on

In [4]:
ROOT.gSystem.Load("libedm4hep")
ROOT.gSystem.Load("libpodio")
ROOT.gSystem.Load("libFCCAnalyses")

0

In [5]:
_edm  = ROOT.edm4hep.ReconstructedParticleData()
_pod  = ROOT.podio.ObjectID()
_fcc  = ROOT.dummyLoader

In [6]:
#df = ROOT.RDataFrame("events", "test_4f_sw_sl_eL_pR_edm4hep.root")
df = ROOT.RDataFrame("events", "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root")

In [7]:
df.Describe()

Dataframe from TChain events in file E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root

Property                Value
--------                -----
Columns in total           39
Columns from defines        0
Event loops run             0
Processing slots            4

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RVec<Float_t>                     Dataset
MCParticles                             ROOT::V

In [14]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticleData> get_enu(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
    ROOT::VecOps::RVec<edm4hep::MCParticleData> res;
    for (size_t i = 0; i <= in.size(); i++) {
        const auto& mc = in[i];
        const auto& mc_next = in[i+1];
        if (abs(mc.PDG) == 11 && abs(mc_next.PDG) == 12) {
            res.push_back(mc);
            res.push_back(mc_next);
            break;
        }
    }
    return res;
}

input_line_164:1:20: error: use of undeclared identifier 'MCParticleData'
ROOT::VecOps::RVec<MCParticleData> get_enu(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
                   ^
input_line_164:2:24: error: use of undeclared identifier 'MCParticleData'
    ROOT::VecOps::RVec<MCParticleData> res;
                       ^
input_line_164:2:40: error: C++ requires a type specifier for all declarations
    ROOT::VecOps::RVec<MCParticleData> res;
                                       ^


In [8]:
df2 = (df
       .Define("Electrons", "MCParticles[MCParticles.generatorStatus == 1 && abs(MCParticles.PDG) == 11]")
       .Define("Electrons_p", "FCCAnalyses::MCParticle::get_p(Electrons)")
       .Define("Electrons_theta", "FCCAnalyses::MCParticle::get_theta(Electrons)")
       .Define("Electrons_n", "Electrons.size()")
       .Define("LeadingElectron_index", "ArgMax(Electrons_p)")
       .Define("LeadingElectron_p", "Electrons_p[LeadingElectron_index]")
       .Define("LeadingElectron_theta", "Electrons_theta[LeadingElectron_index]")
       )

In [9]:
h = {}
h["p"] = df2.Histo1D("Electrons_p")
h["n"] = df2.Histo1D("Electrons_n")
h["lp"] = df2.Histo1D("LeadingElectron_p")
h["ltheta"] = df2.Histo1D("LeadingElectron_theta")

In [10]:
c = {}
for v in h:
    c[v] = ROOT.TCanvas()
    h[v].Draw()
    c[v].Draw()
